In [1]:
from libraries import utils
from libraries import tfim_functions
from libraries import lib
from libraries import training
import torch
import torch.nn as nn
import numpy as np
from libraries.NeuralStates import *
import matplotlib.pyplot as plt
import qutip as qt

In [39]:
N = 6; J = 1; h = 1

def ror(n,rotations,width):
    return (2**width-1)&(n>>rotations|n<<(width-rotations))

In [40]:
bin(ror(7, 1, N))

'0b100011'

In [41]:
distinct_samples = [0, 1, 3, 5, 7, 9, 11, 21]

In [42]:
all_samples = set()
for x in distinct_samples:
    for shift in range(N):
        shifted = ror(x, shift, N)
        all_samples.add(shifted)
        all_samples.add(shifted ^ 0b111111)
print(all_samples)
print(len(all_samples))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63}
64


In [44]:
utils.generate_input_samples(N, distinct_samples)

tensor([[0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0.],
        [1., 0., 1., 0., 1., 0.]])

In [66]:
def generate_binary_necklaces(N):
    a = [0] * (N + 1)
    result = []

    def gen(t, p):
        if t > N:
            if N % p == 0:
                result.append(int(''.join(str(x) for x in a[1:N+1]), 2))
        else:
            a[t] = a[t - p]
            gen(t + 1, p)
            for j in range(a[t - p] + 1, 2):
                a[t] = j
                gen(t + 1, t)

    gen(1, 1)
    return result

# Example usage:
N = 20
necklaces = generate_binary_necklaces(N)
print(len(necklaces))
print(necklaces)
# 

52488
[0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 381, 383, 385, 387, 389, 391, 393, 395, 397, 399, 401, 403, 405, 407, 409, 411, 413, 415, 417, 

In [73]:
def booth_algorithm(s):
    s += s
    n = len(s) // 2
    i = j = k = 0
    while i < n and j < n and k < n:
        if s[i+k] == s[j+k]:
            k += 1
            continue
        elif s[i+k] > s[j+k]:
            i = i + k + 1
            if i <= j:
                i = j + 1
        else:
            j = j + k + 1
            if j <= i:
                j = i + 1
        k = 0
    return min(i, j)

def minimal_rotation(s):
    i = booth_algorithm(s)
    return s[i:] + s[:i]

def bitwise_complement(s):
    return ''.join('1' if c == '0' else '0' for c in s)

def generate_necklaces(n):
    """Generate all binary necklaces of length n (binary strings modulo rotation)."""
    a = [0] * (n + 1)
    necklaces = []

    def gen(t, p):
        if t > n:
            if n % p == 0:
                necklaces.append(''.join(str(x) for x in a[1:n+1]))
        else:
            a[t] = a[t - p]
            gen(t + 1, p)
            for j in range(a[t - p] + 1, 2):
                a[t] = j
                gen(t + 1, t)

    gen(1, 1)
    return necklaces

def generate_rotation_complement_classes(n):
    necklaces = generate_necklaces(n)
    unique_reps = set()

    for s in necklaces:
        comp = bitwise_complement(s)
        comp_min_rot = minimal_rotation(comp)
        rep = min(s, comp_min_rot)
        unique_reps.add(rep)

    return sorted(unique_reps)

# Example usage
n = 4
reps = generate_rotation_complement_classes(n)
print(f"Distinct classes of length {n} modulo rotation + complement:")
print(reps)

Distinct classes of length 4 modulo rotation + complement:
['0000', '0001', '0011', '0101', '0111']


In [85]:
def all_rotations(s):
    return [s[i:] + s[:i] for i in range(len(s))]

def canonical_rep(s):
    comp = bitwise_complement(s)
    rotations_s = all_rotations(s)
    rotations_comp = all_rotations(comp)
    all_candidates = rotations_s + rotations_comp
    return min(all_candidates)

def generate_rotation_complement_classes_bruteforce(n):
    unique_reps = set()
    for i in range(2**n):
        s = format(i, f'0{n}b')
        rep = canonical_rep(s)
        unique_reps.add(rep)
    return sorted(unique_reps)
n = 6
reps = generate_rotation_complement_classes_bruteforce(n)
print(f"Distinct classes of length {n} modulo rotation + complement:")
print(reps)

Distinct classes of length 6 modulo rotation + complement:
['000000', '000001', '000011', '000101', '000111', '001001', '001011', '010101']


In [86]:
[int(n, 2) for n in reps]

[0, 1, 3, 5, 7, 9, 11, 21]

In [68]:
def amp_phase(nn_output):
    return nn_output[:, 0] * torch.exp(1.j * 2 * np.pi * nn_output[:, 1])

In [90]:
def symmetric_TFIM_expectation_from_torch(nn_output, samples, vars, output_to_psi):
    N, J, Gamma = vars
    dim = 2 ** N
    condensed_psi = output_to_psi(nn_output)
    psi = torch.zeros(dim, dtype = torch.complex64)
    for i, x in enumerate(samples):
        for shift in range(N):
            shifted = ror(x, shift, N)
            psi[shifted] = condensed_psi[i]
            complement = shifted ^ (2 ** (N - 1))
            psi[complement] = condensed_psi[i]
    mag2 = torch.norm(psi, 2) ** 2
    # mag = sum(abs(n) ** 2 for n in psi) # make these torch functions, use torch.norm
    phi = torch.zeros(dim, dtype = torch.complex64)
    bra_psi = psi.reshape((1, -1)).conj()
    lib.TFIM_multiply(psi, phi, N, J, Gamma)

    res = (bra_psi @ phi.reshape((-1, 1)))[0][0].real

    return res / mag2

In [ ]:
input = utils.generate_input_samples(N, distinct_samples)
mlp_model = nn.Sequential(
    nn.Linear(N, 5*N),
    nn.Sigmoid(),
    nn.Linear(5*N, 2),
    nn.Sigmoid()
)
mlp_epochs, mlp_loss_data = lib.train_model_to_gs(mlp_model, 
                      lambda model : model(input), 
                      lambda nn_output : symmetric_TFIM_expectation_from_torch(nn_output, distinct_samples, (N, J, h), amp_phase), 
                      1000, data_rate = 1)

In [ ]:
H = lib.TFIM_hamiltonian(N, J, h)
eigs = H.eigenstates()
true_gse = eigs[0][0]
true_gs = eigs[1][0]
mlp_gse = symmetric_TFIM_expectation_from_torch(mlp_model(input), distinct_samples, (N, J, h), amp_phase).item()
print('symmetric results')
print('est', mlp_gse)
print('error', abs((mlp_gse - true_gse) / true_gse))
condensed_gs = amp_phase(mlp_model(input))
mlp_gs = torch.cat((half_gs, half_gs.__reversed__()))
mag = np.sqrt(sum(abs(n) ** 2 for n in mlp_gs).detach().numpy() )
mlp_gs = qt.Qobj(mlp_gs.data / mag)
print('fidelity', qt.fidelity(mlp_gs, true_gs))
errors = abs((np.array(mlp_loss_data) - np.array(len(mlp_loss_data) * [true_gse])) / true_gse)
plt.plot(mlp_epochs, errors)
plt.yscale('log')
plt.title('symmetric')
plt.show()